In [1]:
import pandas as pd
import numpy as np

In [2]:
eur_usd = pd.read_csv('raw data/EUR_USD.csv')
eur_usd['Date'] = pd.to_datetime(eur_usd['Date'])
eur_usd.drop(['Open', 'High', 'Low', 'Vol.', 'Change %'], axis=1, inplace=True)
eur_usd.rename(columns={'Price': 'EUR/USD', 'Date': 'date'}, inplace=True)
print(eur_usd.head())
print(eur_usd.dtypes)

vix = pd.read_csv('raw data/VIX.csv')
vix['Date'] = pd.to_datetime(vix['Date'])
vix = vix[['Date', 'Dernier']]
vix.rename(columns={'Dernier': 'VIX', 'Date': 'date'}, inplace=True)
vix['VIX'] = vix['VIX'].str.replace(',', '.').astype(float)
print(vix.head())
print(vix.dtypes)

euribor = pd.read_csv('raw data/Euribor_90d.csv')
euribor['DATE'] = pd.to_datetime(euribor['DATE'])
euribor = euribor.iloc[:, [0,2]]
euribor.rename(columns = {'DATE': 'date', euribor.columns.values[1]: 'Euribor_90d'}, inplace=True)
print(euribor.head())
print(euribor.dtypes)

        date  EUR/USD
0 2024-12-18   1.0501
1 2024-12-17   1.0489
2 2024-12-16   1.0510
3 2024-12-13   1.0502
4 2024-12-12   1.0467
date       datetime64[ns]
EUR/USD           float64
dtype: object
        date    VIX
0 2024-12-27  17.40
1 2024-12-26  14.73
2 2024-12-24  14.27
3 2024-12-23  16.78
4 2024-12-20  18.36
date    datetime64[ns]
VIX            float64
dtype: object
        date  Euribor_90d
0 1994-01-31         6.91
1 1994-02-28         6.86
2 1994-03-31         6.75
3 1994-04-30         6.57
4 1994-05-31         6.24
date           datetime64[ns]
Euribor_90d           float64
dtype: object


C:\Users\mathi\AppData\Local\Temp\ipykernel_21808\3424196915.py:9: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  vix['Date'] = pd.to_datetime(vix['Date'])


In [3]:
ecb_al = pd.read_csv('raw data/ECB_Assets_liabilities.csv')
ecb_al['DATE'] = pd.to_datetime(ecb_al['DATE'])
ecb_al.drop(['TIME PERIOD'], axis=1, inplace=True)
ecb_al.rename(columns = {"DATE": "date", "Total Assets/Liabilities reported by the ECB (stocks) (BSI.M.4F.N.N.T00.A.1.Z5.0000.Z01.E)":"ECB_assets"}, inplace=True)
print(ecb_al.head())
print(ecb_al.dtypes)

fed_al = pd.read_csv('raw data/Fed_WALCL.csv')
fed_al['observation_date'] = pd.to_datetime(fed_al['observation_date'])
fed_al.rename(columns = {"observation_date": "date", "WALCL": "FED_assets"}, inplace=True)
print(fed_al.head())
print(fed_al.dtypes)

        date  ECB_assets
0 1999-01-31    120372.0
1 1999-02-28    120146.0
2 1999-03-31    108028.0
3 1999-04-30     89502.0
4 1999-05-31     81784.0
date          datetime64[ns]
ECB_assets           float64
dtype: object
        date  FED_assets
0 2002-12-18      719542
1 2002-12-25      732059
2 2003-01-01      730994
3 2003-01-08      723762
4 2003-01-15      720074
date          datetime64[ns]
FED_assets             int64
dtype: object


In [4]:
from functools import reduce
dfs = [ecb_al, fed_al, eur_usd, vix, euribor]
merged_df = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), dfs)
merged_df = merged_df.sort_values(by='date')
print(merged_df.head())

        date  ECB_assets  FED_assets  EUR/USD  VIX  Euribor_90d
0 1994-01-31         NaN         NaN      NaN  NaN         6.91
1 1994-02-28         NaN         NaN      NaN  NaN         6.86
2 1994-03-31         NaN         NaN      NaN  NaN         6.75
3 1994-04-30         NaN         NaN      NaN  NaN         6.57
4 1994-05-31         NaN         NaN      NaN  NaN         6.24


In [7]:
merged_df[['ECB_assets', 'FED_assets', 'EUR/USD', 'VIX']] = merged_df[['ECB_assets', 'FED_assets', 'EUR/USD', 'VIX']].ffill()
merged_df['Euribor_90d'] = merged_df['Euribor_90d'].bfill()
merged_df = merged_df[(merged_df['date'] >= '2008-01-01') & (merged_df['date'] <= '2024-12-01')]
print(merged_df.head(), "\n", merged_df.dtypes)

          date  ECB_assets  FED_assets  EUR/USD    VIX  Euribor_90d
421 2008-01-01    254025.0    890662.0   1.4587    NaN       4.4815
422 2008-01-02    254025.0    922187.0   1.4725  23.17       4.4815
423 2008-01-03    254025.0    922187.0   1.4746  22.49       4.4815
424 2008-01-04    254025.0    922187.0   1.4743  23.94       4.4815
425 2008-01-07    254025.0    922187.0   1.4688  23.79       4.4815 
 date           datetime64[ns]
ECB_assets            float64
FED_assets            float64
EUR/USD               float64
VIX                   float64
Euribor_90d           float64
dtype: object


In [8]:
merged_df.to_csv('daily_data.csv', index=False)

In [7]:
def theoretical_cip(i_ecb, i_fed, spot_rate):
    f = spot_rate*(1+i_ecb)/(1+i_fed)
    return f


In [5]:
def cip_deviation(forward, theoretical_cip):
    deviation = forward - theoretical_cip
    return deviation